Rerfence: https://github.com/ruohoruotsi/LSTM-Music-Genre-Classification/blob/master/lstm_genre_classifier_keras.py
                  https://gist.github.com/parulnith/7f8c174e6ac099e86f0495d3d9a4c01e

In [0]:
!wget http://opihi.cs.uvic.ca/sound/genres.tar.gz

--2019-04-20 16:21:34--  http://opihi.cs.uvic.ca/sound/genres.tar.gz
Resolving opihi.cs.uvic.ca (opihi.cs.uvic.ca)... 142.104.68.135
Connecting to opihi.cs.uvic.ca (opihi.cs.uvic.ca)|142.104.68.135|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1225573614 (1.1G) [application/x-gzip]
Saving to: ‘genres.tar.gz’

genres.tar.gz       100%[===================>]   1.14G  12.4MB/s    in 1m 44s  

2019-04-20 16:23:23 (11.3 MB/s) - ‘genres.tar.gz’ saved [1225573614/1225573614]



In [0]:
!tar xvzf genres.tar.gz

genres/
genres/blues/
genres/blues/blues.00000.au
genres/blues/blues.00001.au
genres/blues/blues.00002.au
genres/blues/blues.00003.au
genres/blues/blues.00004.au
genres/blues/blues.00005.au
genres/blues/blues.00006.au
genres/blues/blues.00007.au
genres/blues/blues.00008.au
genres/blues/blues.00009.au
genres/blues/blues.00010.au
genres/blues/blues.00011.au
genres/blues/blues.00012.au
genres/blues/blues.00013.au
genres/blues/blues.00014.au
genres/blues/blues.00015.au
genres/blues/blues.00016.au
genres/blues/blues.00017.au
genres/blues/blues.00018.au
genres/blues/blues.00019.au
genres/blues/blues.00020.au
genres/blues/blues.00021.au
genres/blues/blues.00022.au
genres/blues/blues.00023.au
genres/blues/blues.00024.au
genres/blues/blues.00025.au
genres/blues/blues.00026.au
genres/blues/blues.00027.au
genres/blues/blues.00028.au
genres/blues/blues.00029.au
genres/blues/blues.00030.au
genres/blues/blues.00031.au
genres/blues/blues.00032.au
genres/blues/blues.00033.au
genres/blues/blues.00034.a

In [0]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
import pathlib
import csv
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import keras
from keras import models
from keras import layers

import warnings
warnings.filterwarnings('ignore')

from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

import timeit


Using TensorFlow backend.


In [0]:
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./genres/{g}'):
        songname = f'./genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [0]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate flatness poly chroma_cqt chroma_cens spectral_contrast' #melspectrogram
for i in range(1, 21):
    header += f' mfcc{i}' 
for i in range(1, 7):
    header += f' tonnetz{i}'    
header += ' label'
header = header.split()

In [0]:
import json

In [0]:
#file = open('blues_test.json', 'w', newline='')
#with file:
#    writer = json.writer(file)
#    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()

with open('blues_test.txt', 'w') as outfile:  
    json.dump(data, outfile)

AttributeError: ignored

In [0]:
start = timeit.default_timer()

rmse = librosa.feature.rmse(y=y)

songname = f'genres/blues/blues.00000.au'

# new features
flatness = librosa.feature.spectral_flatness(y=y)

S = np.abs(librosa.stft(y))
p0 = librosa.feature.poly_features(S=S, order=0)

tonnetz = librosa.feature.tonnetz(y=y, sr=sr)

chroma_cq = librosa.feature.chroma_cqt(y=y, sr=sr)

chroma_cens = librosa.feature.chroma_cens(y=y, sr=sr) 

S = np.abs(librosa.stft(y))
contrast = librosa.feature.spectral_contrast(S=S, sr=sr)

y, sr = librosa.load(songname, mono=True, duration=30)
chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
zcr = librosa.feature.zero_crossing_rate(y)
mfcc = librosa.feature.mfcc(y=y, sr=sr)


to_append = f'{filename} {chroma_stft} {rmse} {spec_cent} {spec_bw} {rolloff} {zcr} {flatness} {p0} {chroma_cq} {chroma_cens} {contrast}'    
for e in mfcc:
    to_append += f' {e}'

for i in tonnetz:
    to_append += f' {i}'
to_append += f' {g}'

with open('blues_test1.txt', 'w') as outfile:  
    json.dump(to_append, outfile)

#file = open('blues_test.txt', 'a', newline='')
#with file:
#    writer = csv.writer(file)
#    writer.writerow(to_append.split())
    
stop = timeit.default_timer()
#print(to_append)
print('Time: ', stop - start)

Time:  3.1241305919998013


In [0]:
ls

blues001_test.csv  blues_test.csv   blues_test.txt  genres.tar.gz  sample_data/
blues_test1.txt    blues_test.json  genres/         img_data/      songs.csv


In [0]:
cd img_data/

/content/img_data


In [0]:
ls

blues/      country/  hiphop/  metal/  reggae/
classical/  disco/    jazz/    pop/    rock/


In [0]:
cd blues/ 

/content/img_data/blues


In [0]:
cd ..

/content


In [0]:
ls

blues00000.png  blues00020.png  blues00040.png  blues00060.png  blues00080.png
blues00001.png  blues00021.png  blues00041.png  blues00061.png  blues00081.png
blues00002.png  blues00022.png  blues00042.png  blues00062.png  blues00082.png
blues00003.png  blues00023.png  blues00043.png  blues00063.png  blues00083.png
blues00004.png  blues00024.png  blues00044.png  blues00064.png  blues00084.png
blues00005.png  blues00025.png  blues00045.png  blues00065.png  blues00085.png
blues00006.png  blues00026.png  blues00046.png  blues00066.png  blues00086.png
blues00007.png  blues00027.png  blues00047.png  blues00067.png  blues00087.png
blues00008.png  blues00028.png  blues00048.png  blues00068.png  blues00088.png
blues00009.png  blues00029.png  blues00049.png  blues00069.png  blues00089.png
blues00010.png  blues00030.png  blues00050.png  blues00070.png  blues00090.png
blues00011.png  blues00031.png  blues00051.png  blues00071.png  blues00091.png
blues00012.png  blues00032.png  blues00052.png  blue

In [0]:
from google.colab import files
files.download('blues_test1.txt') 

When we explore data without mean, it was too big and cannot save in csv file. This is because every feature has huge array and we decided that the best way  is to use mean.

In [0]:
data = pd.read_csv('blues_test1.txt')
data.head()

""


In [0]:
data.shape

(0, 1)